# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 26 2022 9:08AM,238145,10,SNAP1142340,"Snap Medical Industries, LLC",Released
1,May 26 2022 9:08AM,238145,10,SNAP29632228,"Snap Medical Industries, LLC",Released
2,May 26 2022 9:08AM,238145,10,SNAP29643349,"Snap Medical Industries, LLC",Released
3,May 26 2022 9:08AM,238143,15,8063360,"Laser Pharmaceuticals, LLC",Released
4,May 26 2022 9:07AM,238141,15,8062772,"Mizner Bioscience, LLC",Released
5,May 26 2022 9:07AM,238141,15,8062773,"Mizner Bioscience, LLC",Released
6,May 26 2022 9:07AM,238138,15,VT80172827,"Virtus Pharmaceuticals, LLC",Released
7,May 26 2022 9:07AM,238138,15,VT80172828,"Virtus Pharmaceuticals, LLC",Released
8,May 26 2022 9:07AM,238138,15,VT80172829,"Virtus Pharmaceuticals, LLC",Released
9,May 26 2022 9:07AM,238138,15,VT80172830,"Virtus Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,238140,Released,33
29,238141,Released,2
30,238142,Released,1
31,238143,Released,1
32,238145,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
238140,NaN,33.0
238141,NaN,2.0
238142,NaN,1.0
238143,NaN,1.0
238145,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
238021,33.0,1.0
238071,0.0,1.0
238087,0.0,1.0
238090,11.0,18.0
238094,9.0,31.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
238021,33,1
238071,0,1
238087,0,1
238090,11,18
238094,9,31


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,238021,33,1
1,238071,0,1
2,238087,0,1
3,238090,11,18
4,238094,9,31


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,238021,33,1
1,238071,,1
2,238087,,1
3,238090,11,18
4,238094,9,31


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 26 2022 9:08AM,238145,10,"Snap Medical Industries, LLC"
3,May 26 2022 9:08AM,238143,15,"Laser Pharmaceuticals, LLC"
4,May 26 2022 9:07AM,238141,15,"Mizner Bioscience, LLC"
6,May 26 2022 9:07AM,238138,15,"Virtus Pharmaceuticals, LLC"
11,May 26 2022 9:04AM,238137,15,"Carwin Pharmaceutical Associates, LLC"
22,May 26 2022 9:02AM,238136,15,"Brookfield Pharmaceuticals, LLC"
23,May 26 2022 9:02AM,238135,15,"Alliance Pharma, Inc."
27,May 26 2022 8:58AM,238140,10,ISDIN Corporation
60,May 26 2022 8:56AM,238142,10,"Amcyte Pharma, Inc."
61,May 26 2022 8:32AM,238134,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,May 26 2022 9:08AM,238145,10,"Snap Medical Industries, LLC",,3
1,May 26 2022 9:08AM,238143,15,"Laser Pharmaceuticals, LLC",,1
2,May 26 2022 9:07AM,238141,15,"Mizner Bioscience, LLC",,2
3,May 26 2022 9:07AM,238138,15,"Virtus Pharmaceuticals, LLC",,5
4,May 26 2022 9:04AM,238137,15,"Carwin Pharmaceutical Associates, LLC",,11
5,May 26 2022 9:02AM,238136,15,"Brookfield Pharmaceuticals, LLC",,1
6,May 26 2022 9:02AM,238135,15,"Alliance Pharma, Inc.",,4
7,May 26 2022 8:58AM,238140,10,ISDIN Corporation,,33
8,May 26 2022 8:56AM,238142,10,"Amcyte Pharma, Inc.",,1
9,May 26 2022 8:32AM,238134,10,Emerginnova,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,May 26 2022 9:08AM,238145,10,"Snap Medical Industries, LLC",3,
1,May 26 2022 9:08AM,238143,15,"Laser Pharmaceuticals, LLC",1,
2,May 26 2022 9:07AM,238141,15,"Mizner Bioscience, LLC",2,
3,May 26 2022 9:07AM,238138,15,"Virtus Pharmaceuticals, LLC",5,
4,May 26 2022 9:04AM,238137,15,"Carwin Pharmaceutical Associates, LLC",11,
5,May 26 2022 9:02AM,238136,15,"Brookfield Pharmaceuticals, LLC",1,
6,May 26 2022 9:02AM,238135,15,"Alliance Pharma, Inc.",4,
7,May 26 2022 8:58AM,238140,10,ISDIN Corporation,33,
8,May 26 2022 8:56AM,238142,10,"Amcyte Pharma, Inc.",1,
9,May 26 2022 8:32AM,238134,10,Emerginnova,5,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 26 2022 9:08AM,238145,10,"Snap Medical Industries, LLC",3,
1,May 26 2022 9:08AM,238143,15,"Laser Pharmaceuticals, LLC",1,
2,May 26 2022 9:07AM,238141,15,"Mizner Bioscience, LLC",2,
3,May 26 2022 9:07AM,238138,15,"Virtus Pharmaceuticals, LLC",5,
4,May 26 2022 9:04AM,238137,15,"Carwin Pharmaceutical Associates, LLC",11,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,May 26 2022 9:08AM,238145,10,"Snap Medical Industries, LLC",3.0,NaN
1,May 26 2022 9:08AM,238143,15,"Laser Pharmaceuticals, LLC",1.0,NaN
2,May 26 2022 9:07AM,238141,15,"Mizner Bioscience, LLC",2.0,NaN
3,May 26 2022 9:07AM,238138,15,"Virtus Pharmaceuticals, LLC",5.0,NaN
4,May 26 2022 9:04AM,238137,15,"Carwin Pharmaceutical Associates, LLC",11.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 26 2022 9:08AM,238145,10,"Snap Medical Industries, LLC",3.0,0.0
1,May 26 2022 9:08AM,238143,15,"Laser Pharmaceuticals, LLC",1.0,0.0
2,May 26 2022 9:07AM,238141,15,"Mizner Bioscience, LLC",2.0,0.0
3,May 26 2022 9:07AM,238138,15,"Virtus Pharmaceuticals, LLC",5.0,0.0
4,May 26 2022 9:04AM,238137,15,"Carwin Pharmaceutical Associates, LLC",11.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1428751,44.0,0.0
15,1428830,24.0,0.0
16,714351,2.0,1.0
18,476265,13.0,0.0
19,952453,23.0,19.0
20,952306,70.0,53.0
21,476246,2.0,0.0
22,476226,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1428751,44.0,0.0
1,15,1428830,24.0,0.0
2,16,714351,2.0,1.0
3,18,476265,13.0,0.0
4,19,952453,23.0,19.0
5,20,952306,70.0,53.0
6,21,476246,2.0,0.0
7,22,476226,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,44.0,0.0
1,15,24.0,0.0
2,16,2.0,1.0
3,18,13.0,0.0
4,19,23.0,19.0
5,20,70.0,53.0
6,21,2.0,0.0
7,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,44.0
1,15,Released,24.0
2,16,Released,2.0
3,18,Released,13.0
4,19,Released,23.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Executing,0.0,0.0,1.0,0.0,19.0,53.0,0.0,0.0
Released,44.0,24.0,2.0,13.0,23.0,70.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Executing,0.0,0.0,1.0,0.0,19.0,53.0,0.0,0.0
1,Released,44.0,24.0,2.0,13.0,23.0,70.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Executing,0.0,0.0,1.0,0.0,19.0,53.0,0.0,0.0
1,Released,44.0,24.0,2.0,13.0,23.0,70.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()